# Imports

In [177]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Importing and Splitting data

In [198]:
file = pd.read_csv("audi.csv", quotechar='"', skipinitialspace=True)

modelEncoder = LabelEncoder()
modelEncoder.fit(file["model"])
file["model"] = modelEncoder.transform(file["model"])

transmissionEncoder = LabelEncoder()
transmissionEncoder.fit(file["transmission"])
file["transmission"] = transmissionEncoder.transform(file["transmission"])

fuelTypeEncoder = LabelEncoder()
fuelTypeEncoder.fit(file["fuelType"])
file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

file = file.head(1000)
# X = file.drop(columns = ['price'])
# Y = file.price
# print(file)

# X = file.drop(['price'], axis = 1).to_numpy()
Y = file['price'].values.reshape(-1,1)
X = file[['year', 'mileage', 'tax', 'mpg', 'engineSize']].to_numpy(dtype = 'int')

print(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)

[[ 2017 15735   150    55     1]
 [ 2016 36203    20    64     2]
 [ 2016 29946    30    55     1]
 ...
 [ 2019  7461   150    37     3]
 [ 2017 37492   150    34     3]
 [ 2018 36715   150    35     3]]


# Using Scikit-Learn

In [18]:
myTree = DecisionTreeRegressor(random_state = 601)
myTree.fit(X_train, Y_train)
print("R Squared: ", myTree.score(X_test, Y_test))

R Squared:  0.8469942517214841


# Visualising Decsion Tree:

In [181]:
treeView = DecisionTreeRegressor(random_state = 601, max_depth = 3)
treeView.fit(X_train,Y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(8,3), dpi = 150)
plot_tree(treeView, feature_names = X.columns, filled=True);

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

<Figure size 1200x450 with 0 Axes>

# Entropy

In [6]:
# def calc_total_entropy(train_data, label, class_list):
#     total_row = train_data.shape[0]
#     total_entr = 0 #l
    
#     for c in class_list: 
#         total_class_count = train_data[train_data[label] == c].shape[0] 
#         total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) 
#         total_entr += total_class_entr 
    
#     return total_entr



#  def entropy(self, y):
#         entropy = 0
#         labels = np.unique(y)
#         for label in labels:
#             label_examples = y[y == label]
#             pl = len(label_examples) / len(y)
#             entropy += -pl * np.log2(pl)
#         return entropy

# RMSE

In [182]:
def rmse(h, y):
  return sqrt(mean_squared_error(h, y))

In [309]:
class Node():
    def __init__(self, feature = None, threshold = None, left = None, right = None, gain = None, value=None):
        
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        
        self.gain = gain

        self.value = value #leaf

In [331]:
class DTRegressor():
    def __init__(self, minSamples = 2, maxDepth = 2):
        self.root = None
        self.minSamples = minSamples
        self.maxDepth = maxDepth
        
    def entropy(self, y): #need to change method
        entropy = 0
        
        labels = np.unique(y)
        # loop over each unique value
        for label in labels:
            # find elements in y that are equal to that label
            label_examples = y[y == label]
            # get ratio of the label
            pl = len(label_examples) / len(y)
            # calculate entropy
            entropy += -pl * np.log2(pl)
        return entropy
        
    def infoGain(self, parent, leftNode, rightNode): #need to checkkk
#         print("HERE 5")
        leftWeight = len(leftNode) / len(parent)
        rightWeight = len(rightNode) / len(parent)
        
        information_gain = np.var(parent) - (leftWeight * np.var(leftNode) + rightWeight * np.var(rightNode))


#         parent_entropy = self.entropy(parent)
#         entropy_left, entropy_right = self.entropy(leftNode), self.entropy(rightNode)
#         # calculate weighted entropy 
#         weighted_entropy = leftWeight * entropy_left + rightWeight * entropy_right
#         # calculate information gain 
#         information_gain = parent_entropy - weighted_entropy
# #         print(information_gain)
        return information_gain
        
    def splitTree(self, dataset, feature, threshold):
#         print("HERE 4")
        leftBranch = []
        rightBranch = []
        for i in dataset:
            if i[feature] <= threshold:
                leftBranch.append(i)
            else:
                rightBranch.append(i)
        rightBranch = np.array(rightBranch)
        leftBranch = np.array(leftBranch)
        return leftBranch, rightBranch
        
    def bestSplit(self, dataset, X):
#         print("HERE 3")
        bestSplitt = {} 
        maxGain = -1
        for feature in range(X.shape[1]): #feature_index = i
            featureValues = dataset[:, feature] #current feature selected
            thresholds = np.unique(featureValues)
            for j in thresholds: #threshold = j
                # get current split
                leftSide, rightSide = self.splitTree(dataset, feature, j) #splits tree into 2 branches
                if (len(leftSide) > 0 and len(rightSide) > 0 ):
                    parent = dataset[:, -1]
                    leftNode = leftSide[:, -1]
                    rightNode = rightSide[:, -1]

                    currentGain = self.infoGain(parent, leftNode, rightNode) 
                    if currentGain > maxGain:
                        
                        bestSplitt["feature"] = feature
                        bestSplitt["thresholds"] = j
                        bestSplitt["leftSide"] = leftSide
                        bestSplitt["rightSide"] = rightSide
                        bestSplitt["gain"] = currentGain
                        maxGain = currentGain
                
        return bestSplitt
   
        
    def treeBuild(self, dataset, currentDepth = 0):
#         print("HERE 2")
        X = dataset[:,:-1]
        Y = dataset[:,-1] #Split training into features and labels
        samplesNumb = X.shape[0]
        featuresNumb = X.shape[1]
        
        #iterates until this condition is met
        if X.shape[0] >= self.minSamples and currentDepth <= self.maxDepth:
#             bestSplit = self.bestSplit(dataset, samplesNumb, featuresNumb)
            bestSplit = self.bestSplit(dataset, X)
            
            if bestSplit["gain"] > 0:
                leftTree = self.treeBuild(bestSplit["leftSide"], currentDepth + 1)
                rightTree = self.treeBuild(bestSplit["rightSide"], currentDepth + 1)
                decision = Node(bestSplit["feature"], bestSplit["thresholds"], leftTree, rightTree, bestSplit["gain"])
                
                return decision
            
        leafValue = np.mean(Y) #calculates mean of leaf nodes
        val = Node(value = leafValue)
        return val
    
    def predictionLoop(self, testRow, root):
        if root.value != None: #not empty
            return root.value
        
        featureVal = testRow[root.feature]
        print(featureVal)
        if featureVal <= root.threshold:
            return self.predictionLoop(testRow, root.left)
        else:
            return self.predictionLoop(testRow, root.right)
        
   
    def predict(self, xTest):
        predictions = []
        for row in xTest:
#             print(row)
            predictions.append(self.predictionLoop(row, self.root)) 
        return predictions

        
        
    def fit(self, X, Y):
#         print("HERE 1")
        dataset = np.concatenate((X, Y), axis=1) #Joins training data back together
        self.root = self.treeBuild(dataset)

In [330]:
myTree = DTRegressor(3, 3)  
myTree.fit(X_train, Y_train) #trains the model using the training set


testy = [[2016,49050,325,29.4,4.0]] # using this for prediction £44985
print(myTree.predict(testy)) 

29.4
4.0
29.4
2016
[39326.666666666664]


[43814245.71274468]


In [167]:
def evaluation(train, test, yTrain, yTest):    
    y_pred = myTree.predict(X_test)
    error = rmse(yTest, y_pred) 
    print('RMSE value for k is:', error)
    for i in range(len(y_pred)):
        print("\nOriginal value:", yTest[i], "vs the predicted value:", y_pred[i])
        print("The difference is:", yTest[i] - y_pred[i])